In [1]:
import pandas as pd
from helpers import *
import random

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
file_name = "../../input-data/queries.csv"
queries_df = pd.read_csv(file_name)
print(queries_df.info())
queries_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   config_id  45 non-null     object
 1   config     45 non-null     object
 2   city       45 non-null     object
 3   query_v    45 non-null     object
dtypes: object(4)
memory usage: 1.5+ KB
None


,config_id,config,city,query_v
0,c_p_143_pop_high_hard,"{'p_id': 'p_143', 'persona': 'A former printin...","['Zurich', 'Warsaw', 'Vienna', 'Valencia', 'To...","""Recommend European cities for a high-budget t..."
1,c_p_15_pop_high_hard,"{'p_id': 'p_15', 'persona': 'A patient who see...","['Zurich', 'Vienna', 'Stuttgart', 'Stockholm',...","""High budget European cities to visit in Decem..."
2,c_p_5_pop_high_hard,"{'p_id': 'p_5', 'persona': 'an average Ukraini...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...","""Popular and expensive European cities to visi..."
3,c_p_155_pop_high_hard,"{'p_id': 'p_155', 'persona': 'a digital transf...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...","""European city breaks in September with high-e..."
4,c_p_116_pop_high_hard,"{'p_id': 'p_116', 'persona': 'An entrepreneuri...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...",High budget European city breaks in October wi...


In [4]:
kb = pd.read_csv("../../input-data/kb/eu-cities-database.csv")[["city", "popularity"]]
kb.drop_duplicates(inplace=True, ignore_index=True)
print(len(kb))
kb.head()

200


,city,popularity
0,Aalborg,Medium
1,Adana,Low
2,Adiyaman,Low
3,Agri,Low
4,Amsterdam,High


In [5]:
pop_scores = pd.read_csv("../../data/european-city-data/computed/popularity/popularity_scores.csv")[["city", "weighted_pop_score"]]
pop_scores
kb = kb.merge(pop_scores, on="city", how="right")
kb.rename(columns={"weighted_pop_score": "pop_score"}, inplace=True)
kb.sort_values(by="city", ignore_index=True)

,city,popularity,pop_score
0,Aalborg,Medium,0.041
1,Adana,Low,0.084
2,Adiyaman,Low,0.001
3,Agri,Low,0.009
4,Amsterdam,High,0.425
...,...,...,...
195,Zaporizhzhia,Low,0.006
196,Zaragoza,Medium,0.196
197,Zielona Gora,Low,0.006
198,Zilina,Medium,0.019


In [6]:
def get_top_pop(df, k =10):
    return df.sort_values(by="pop_score", ascending=False).head(k)["city"].tolist()


In [7]:
top_pop_cities = get_top_pop(kb, k=10)
top_pop_cities


['London',
 'Paris',
 'Rome',
 'Barcelona',
 'Moscow',
 'Madrid',
 'Amsterdam',
 'Istanbul',
 'Prague',
 'Dublin']

In [9]:
print(f"Entropy topPop: {compute_normalized_entropy(top_pop_cities)[1]}")
print(f"Gini topPop: {gini_index(top_pop_cities)}")

Entropy topPop: 0.94
Gini topPop: 0.34423076923076923


In [8]:
print(f"Entropy topPop: {compute_entropy_topPop(top_pop_cities)}")
print(f"Gini topPop: {gini_index(top_pop_cities)}")

Entropy topPop: 0.9863
Gini topPop: 0.34423076923076923


In [29]:
queries_df["top_pop_baseline"] = [top_pop_cities] * len(queries_df)


queries_df['relevance'] = queries_df.apply(
    lambda row: get_city_relevance_score(response=row["top_pop_baseline"], query=row["query_v"],
                                         is_baseline=True),
    axis=1
)
print(f"Relevance topPop: {queries_df.relevance.mean()}")

Relevance topPop: 0.7098148148148148


In [30]:
queries_df

,config_id,config,city,query_v,randomPop_nonInformed,top_pop_baseline,relevance
0,c_p_143_pop_high_hard,"{'p_id': 'p_143', 'persona': 'A former printin...","['Zurich', 'Warsaw', 'Vienna', 'Valencia', 'To...","""Recommend European cities for a high-budget t...","[Tallinn, Bordeaux, Milan, London, Naples, Wro...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.925000
1,c_p_15_pop_high_hard,"{'p_id': 'p_15', 'persona': 'A patient who see...","['Zurich', 'Vienna', 'Stuttgart', 'Stockholm',...","""High budget European cities to visit in Decem...","[Podgorica, Szczecin, Chisinau, Cheboksary, Br...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.825000
2,c_p_5_pop_high_hard,"{'p_id': 'p_5', 'persona': 'an average Ukraini...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...","""Popular and expensive European cities to visi...","[Dublin, Saint Petersburg, Cork, Tbilisi, Pari...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.900000
3,c_p_155_pop_high_hard,"{'p_id': 'p_155', 'persona': 'a digital transf...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...","""European city breaks in September with high-e...","[Kirov, Astrakhan, Zaragoza, Bern, Stavanger, ...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.900000
4,c_p_116_pop_high_hard,"{'p_id': 'p_116', 'persona': 'An entrepreneuri...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...",High budget European city breaks in October wi...,"[Toulouse, Strasbourg, Mus, Lyon, Baku, Rome, ...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.900000
5,c_p_162_pop_high_medium,"{'p_id': 'p_162', 'persona': 'An experienced L...","['Zurich', 'Zagreb', 'Volgograd', 'Vilnius', '...","""European cities with vibrant nightlife and po...","[Penza, Plovdiv, Rennes, Ancona, Stavanger, St...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.866667
6,c_p_113_pop_high_medium,"{'p_id': 'p_113', 'persona': 'a Jewish politic...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...","""European city breaks with high-end cultural e...","[Geneva, Perm, Moscow, Belgrade, Vienna, Stutt...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.866667
7,c_p_130_pop_high_medium,"{'p_id': 'p_130', 'persona': 'A fervent collec...","['Zurich', 'Warsaw', 'Vienna', 'Valencia', 'To...",What are some European cities with high-end re...,"[Trabzon, Burgas, Adana, Diyarbakir, Pristina,...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.900000
8,c_p_131_pop_high_medium,"{'p_id': 'p_131', 'persona': 'A metal supplier...","['Zurich', 'Zagreb', 'Volgograd', 'Vilnius', '...",What are some European cities with great night...,"[Perm, Kazan, Volgograd, Lyon, Bordeaux, Porto...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.866667
9,c_p_94_pop_high_medium,"{'p_id': 'p_94', 'persona': 'A popular Twitch ...","['Zagreb', 'Tirana', 'Tbilisi', 'Sofia', 'Sama...","""Looking for a budget-friendly European city b...","[Orenburg, Erfurt, Coimbra, Magdeburg, Logrono...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.633333


In [31]:
queries_df['filters'] = queries_df["query_v"].apply(get_filters_from_query)
queries_df

,config_id,config,city,query_v,randomPop_nonInformed,top_pop_baseline,relevance,filters
0,c_p_143_pop_high_hard,"{'p_id': 'p_143', 'persona': 'A former printin...","['Zurich', 'Warsaw', 'Vienna', 'Valencia', 'To...","""Recommend European cities for a high-budget t...","[Tallinn, Bordeaux, Milan, London, Naples, Wro...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.925000,"{'popularity': 'high', 'month': 'February', 'b..."
1,c_p_15_pop_high_hard,"{'p_id': 'p_15', 'persona': 'A patient who see...","['Zurich', 'Vienna', 'Stuttgart', 'Stockholm',...","""High budget European cities to visit in Decem...","[Podgorica, Szczecin, Chisinau, Cheboksary, Br...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.825000,"{'popularity': 'high', 'month': 'December', 'i..."
2,c_p_5_pop_high_hard,"{'p_id': 'p_5', 'persona': 'an average Ukraini...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...","""Popular and expensive European cities to visi...","[Dublin, Saint Petersburg, Cork, Tbilisi, Pari...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.900000,"{'popularity': 'high', 'budget': 'high', 'mont..."
3,c_p_155_pop_high_hard,"{'p_id': 'p_155', 'persona': 'a digital transf...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...","""European city breaks in September with high-e...","[Kirov, Astrakhan, Zaragoza, Bern, Stavanger, ...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.900000,"{'popularity': 'high', 'interests': 'Outdoors ..."
4,c_p_116_pop_high_hard,"{'p_id': 'p_116', 'persona': 'An entrepreneuri...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...",High budget European city breaks in October wi...,"[Toulouse, Strasbourg, Mus, Lyon, Baku, Rome, ...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.900000,"{'popularity': 'high', 'month': 'October', 'in..."
5,c_p_162_pop_high_medium,"{'p_id': 'p_162', 'persona': 'An experienced L...","['Zurich', 'Zagreb', 'Volgograd', 'Vilnius', '...","""European cities with vibrant nightlife and po...","[Penza, Plovdiv, Rennes, Ancona, Stavanger, St...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.866667,"{'popularity': 'high', 'interests': 'Nightlife..."
6,c_p_113_pop_high_medium,"{'p_id': 'p_113', 'persona': 'a Jewish politic...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...","""European city breaks with high-end cultural e...","[Geneva, Perm, Moscow, Belgrade, Vienna, Stutt...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.866667,"{'popularity': 'high', 'budget': 'high', 'inte..."
7,c_p_130_pop_high_medium,"{'p_id': 'p_130', 'persona': 'A fervent collec...","['Zurich', 'Warsaw', 'Vienna', 'Valencia', 'To...",What are some European cities with high-end re...,"[Trabzon, Burgas, Adana, Diyarbakir, Pristina,...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.900000,"{'popularity': 'high', 'interests': 'Food', 'b..."
8,c_p_131_pop_high_medium,"{'p_id': 'p_131', 'persona': 'A metal supplier...","['Zurich', 'Zagreb', 'Volgograd', 'Vilnius', '...",What are some European cities with great night...,"[Perm, Kazan, Volgograd, Lyon, Bordeaux, Porto...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.866667,"{'popularity': 'high', 'interests': 'Nightlife..."
9,c_p_94_pop_high_medium,"{'p_id': 'p_94', 'persona': 'A popular Twitch ...","['Zagreb', 'Tirana', 'Tbilisi', 'Sofia', 'Sama...","""Looking for a budget-friendly European city b...","[Orenburg, Erfurt, Coimbra, Magdeburg, Logrono...","[London, Paris, Rome, Barcelona, Moscow, Madri...",0.633333,"{'popularity': 'high', 'budget': 'low', 'inter..."
